In [4]:
import json
import numpy as np

In [6]:
# Fungsi untuk memuat file JSON
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

# Muat multimodal features yang sudah disimpan
multimodal_path = "../2. agregation embed/multimodal_features.json"  # sesuaikan path-nya
multimodal_features = load_json(multimodal_path)

In [7]:
db_features = {name: np.array(vec) for name, vec in multimodal_features.items()}

In [8]:
# Fungsi untuk menghitung cosine similarity antara dua vektor
def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

In [11]:
# Fungsi dummy untuk menghasilkan embedding dari query teks
def get_text_embedding(query_text):
    # Di sini, sebagai contoh, kita menghasilkan vektor acak berdimensi 256.
    # Gantilah fungsi ini dengan pemrosesan teks menggunakan model NLP yang sesuai.
    return (np.random.rand(256) - 0.5).astype(np.float32)

In [12]:
# Fungsi dummy untuk menghasilkan embedding dari query gambar
def get_image_embedding(query_image_path):
    # Di sini, sebagai contoh, kita menghasilkan vektor acak berdimensi 256.
    # Gantilah dengan pipeline ekstraksi fitur gambar (misalnya, CNN + pooling).
    return (np.random.rand(256) - 0.5).astype(np.float32)



In [13]:
# Fungsi untuk menggabungkan embedding teks dan gambar (misalnya, melalui konkatenasi)
def fuse_embeddings(text_emb, image_emb):
    return np.concatenate([text_emb, image_emb], axis=0)

In [16]:
# Proses input query dari user
query_type = input("Masukkan tipe query (text, image, both): ").strip().lower()

In [22]:
if query_type == "text":
    query_text = input("Masukkan query teks: ")
    text_emb = get_text_embedding(query_text)
    # Untuk memastikan dimensi query sama dengan database multimodal (512),
    # kita gabungkan dengan vektor nol untuk placeholder gambar.
    zero_image_emb = np.zeros_like(text_emb)  # vektor nol 256-dimensi
    query_embedding = fuse_embeddings(text_emb, zero_image_emb)
elif query_type == "image":
    query_image_path = input("Masukkan path gambar query: ")
    query_embedding = get_image_embedding(query_image_path)
    # Jika database menggunakan multimodal embedding (512-dimensi),
    # kita bisa menggabungkan dengan vektor nol untuk placeholder teks.
    zero_text_emb = np.zeros_like(query_embedding)
    query_embedding = fuse_embeddings(zero_text_emb, query_embedding)
elif query_type == "both":
    query_text = input("Masukkan query teks: ")
    query_image_path = input("Masukkan path gambar query: ")
    text_emb = get_text_embedding(query_text)
    image_emb = get_image_embedding(query_image_path)
    query_embedding = fuse_embeddings(text_emb, image_emb)
else:
    print("Tipe query tidak valid.")
    query_embedding = None

# Proses IR jika query_embedding valid
if query_embedding is not None:
    similarities = {}
    for name, emb in db_features.items():
        sim = cosine_similarity(query_embedding, emb)
        similarities[name] = sim

    # Urutkan hasil berdasarkan similarity tertinggi
    sorted_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    
    top_k = 5
    print(f"\nTop {top_k} hasil pencarian berdasarkan similarity:")
    for i, (name, sim) in enumerate(sorted_results[:top_k]):
        print(f"{i+1}. {name} - Similarity: {sim:.4f}")


Top 5 hasil pencarian berdasarkan similarity:
1. Batik Cendrawasih - Similarity: -0.0069
2. Batik Priangan - Similarity: -0.0085
3. Batik Pekalongan - Similarity: -0.0087
4. Batik Megamendung - Similarity: -0.0189
5. Batik Sidoluhur - Similarity: -0.0242
